# Programación Dinámica en Reinforcement Learning

**Tutorial Completo: De la Teoría a la Práctica**

---

## Índice

1. [Introducción Teórica](#1-introducción-teórica)
2. [Fundamentos Matemáticos](#2-fundamentos-matemáticos)
3. [Explicaciones Intuitivas](#3-explicaciones-intuitivas)
4. [Policy Iteration](#4-policy-iteration)
5. [Value Iteration](#5-value-iteration)
6. [Comparación de Algoritmos](#6-comparación-de-algoritmos)
7. [Experimentos y Casos de Prueba](#7-experimentos-y-casos-de-prueba)
8. [Visualizaciones Avanzadas](#8-visualizaciones-avanzadas)
9. [Ejercicios Prácticos](#9-ejercicios-prácticos)

---

## 1. Introducción Teórica

### ¿Qué es Programación Dinámica?

La **Programación Dinámica (DP)** es una familia de algoritmos que pueden utilizarse para calcular políticas óptimas dado un modelo perfecto del entorno como un **Proceso de Decisión de Markov (MDP)**.

### Características Clave:

1. **Requiere modelo completo**: Necesitamos conocer las probabilidades de transición $p(s',r|s,a)$
2. **Resuelve el problema de forma exacta**: Encuentra la política óptima (no aproximada)
3. **Computacionalmente costosa**: Para espacios de estados grandes
4. **Base teórica**: Fundamento para métodos model-free (Q-learning, SARSA, etc.)

### Aplicaciones:

- Problemas de navegación (GridWorld, laberintos)
- Gestión de inventario
- Control de procesos industriales
- Juegos con estados discretos
- Planificación de trayectorias en robótica

## 2. Fundamentos Matemáticos

### 2.1 Procesos de Decisión de Markov (MDP)

Un MDP se define por la tupla $(\mathcal{S}, \mathcal{A}, P, R, \gamma)$:

- $\mathcal{S}$: Conjunto de estados
- $\mathcal{A}$: Conjunto de acciones
- $P$: Función de transición $P(s'|s,a)$
- $R$: Función de recompensa $R(s,a,s')$
- $\gamma \in [0,1]$: Factor de descuento

### 2.2 Funciones de Valor

#### Función de Valor Estado $V^\pi(s)$:

$$V^\pi(s) = \mathbb{E}_\pi\left[\sum_{t=0}^{\infty} \gamma^t R_{t+1} \mid S_0 = s\right]$$

Es el retorno esperado comenzando desde el estado $s$ y siguiendo la política $\pi$.

#### Función de Valor Acción $Q^\pi(s,a)$:

$$Q^\pi(s,a) = \mathbb{E}_\pi\left[\sum_{t=0}^{\infty} \gamma^t R_{t+1} \mid S_0 = s, A_0 = a\right]$$

Es el retorno esperado tomando la acción $a$ en el estado $s$ y luego siguiendo $\pi$.

### 2.3 Ecuaciones de Bellman

#### Ecuación de Bellman para $V^\pi$:

$$V^\pi(s) = \sum_{a} \pi(a|s) \sum_{s',r} p(s',r|s,a)[r + \gamma V^\pi(s')]$$

Esta ecuación expresa la relación recursiva entre el valor de un estado y los valores de sus sucesores.

#### Ecuación de Optimalidad de Bellman para $V^*$:

$$V^*(s) = \max_a \sum_{s',r} p(s',r|s,a)[r + \gamma V^*(s')]$$

#### Ecuación de Optimalidad de Bellman para $Q^*$:

$$Q^*(s,a) = \sum_{s',r} p(s',r|s,a)\left[r + \gamma \max_{a'} Q^*(s',a')\right]$$

### 2.4 Principio de Optimalidad de Bellman

> **Teorema**: Una política $\pi$ es óptima si y solo si, para todos los estados $s$:
> $$\pi(s) = \arg\max_a Q^*(s,a)$$

Este principio garantiza que existe al menos una política óptima determinista.

## 3. Explicaciones Intuitivas

### 3.1 Analogía: El Viajero en una Ciudad

Imagina que eres un viajero en una ciudad desconocida:

- **Estados ($s$)**: Tu ubicación actual (intersecciones)
- **Acciones ($a$)**: Direcciones que puedes tomar (norte, sur, este, oeste)
- **Recompensas ($r$)**: Tiempo que tardas en moverte (negativo = costo)
- **Objetivo**: Llegar al hotel lo más rápido posible

**Policy Iteration** es como:
1. Tener un plan inicial (política)
2. Evaluar cuánto tiempo tomaría seguir ese plan desde cada ubicación
3. Mejorar el plan eligiendo mejores direcciones
4. Repetir hasta que el plan no pueda mejorar

**Value Iteration** es como:
1. Marcar cada intersección con "tiempo estimado al hotel"
2. Actualizar estas estimaciones mirando las intersecciones vecinas
3. Repetir hasta que las estimaciones se estabilicen
4. Al final, elegir la dirección que lleva a la intersección con menor tiempo

### 3.2 Visualización del GridWorld

```
┌─────┬─────┬─────┬─────┐
│  S  │     │     │     │  S = Estado inicial
├─────┼─────┼─────┼─────┤  G = Meta (Goal)
│     │     │     │     │  # = Obstáculo
├─────┼─────┼─────┼─────┤
│     │  #  │     │     │  
├─────┼─────┼─────┼─────┤
│     │     │     │  G  │
└─────┴─────┴─────┴─────┘
```

En este ejemplo:
- El agente comienza en la esquina superior izquierda
- Debe llegar a la esquina inferior derecha
- Cada movimiento tiene un pequeño costo (-0.01)
- Alcanzar la meta da recompensa +1.0

## 4. Policy Iteration

### 4.1 Algoritmo

Policy Iteration alterna entre dos pasos:

**1. Policy Evaluation (Evaluación de Política)**
```
Repetir hasta convergencia:
    Para cada estado s:
        V(s) ← Σ_{s',r} p(s',r|s,π(s))[r + γV(s')]
```

**2. Policy Improvement (Mejora de Política)**
```
Para cada estado s:
    π'(s) ← argmax_a Σ_{s',r} p(s',r|s,a)[r + γV(s')]
```

**3. Criterio de Parada**
```
Si π' = π, entonces STOP (política óptima encontrada)
Sino, π ← π' y volver a paso 1
```

### 4.2 Implementación Completa

In [ ]:
# Configuración inicial
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import FancyArrowPatch
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd

# Agregar el path del repositorio para importar las implementaciones
repo_path = '/home/user/Reinforcement-learning-guide'
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

# Importar las implementaciones de Dynamic Programming
# Nota: Los módulos están en 02_algoritmos_clasicos/dynamic_programming/
import importlib.util

# Cargar policy_iteration.py
pi_spec = importlib.util.spec_from_file_location(
    "policy_iteration", 
    os.path.join(repo_path, "02_algoritmos_clasicos/dynamic_programming/policy_iteration.py")
)
pi_module = importlib.util.module_from_spec(pi_spec)
pi_spec.loader.exec_module(pi_module)

# Cargar value_iteration.py
vi_spec = importlib.util.spec_from_file_location(
    "value_iteration",
    os.path.join(repo_path, "02_algoritmos_clasicos/dynamic_programming/value_iteration.py")
)
vi_module = importlib.util.module_from_spec(vi_spec)
vi_spec.loader.exec_module(vi_module)

# Importar las clases y funciones
PolicyIteration = pi_module.PolicyIteration
create_gridworld_mdp = pi_module.create_gridworld_mdp
ValueIteration = vi_module.ValueIteration

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Módulos importados correctamente")
print(f"  - PolicyIteration desde: 02_algoritmos_clasicos/dynamic_programming/policy_iteration.py")
print(f"  - ValueIteration desde: 02_algoritmos_clasicos/dynamic_programming/value_iteration.py")

### 4.3 Crear el Entorno GridWorld

In [ ]:
# Crear GridWorld 4x4
print("Creando entorno GridWorld 4x4...")
print("="*50)

transition_probs, rewards, n_states, n_actions = create_gridworld_mdp(
    grid_size=4,
    goal_reward=1.0,
    step_reward=-0.01
)

print(f"✓ Estados: {n_states}")
print(f"✓ Acciones: {n_actions} (0=↑, 1=→, 2=↓, 3=←)")
print(f"✓ Estado inicial: 0 (esquina superior izquierda)")
print(f"✓ Estado objetivo: {n_states-1} (esquina inferior derecha)")
print(f"\nDimensiones de las matrices:")
print(f"  - Transition probabilities: {transition_probs.shape}")
print(f"  - Rewards: {rewards.shape}")

### 4.4 Ejecutar Policy Iteration

In [ ]:
print("\n" + "="*50)
print("EJECUTANDO POLICY ITERATION")
print("="*50 + "\n")

# Crear solver
pi_solver = PolicyIteration(
    n_states=n_states,
    n_actions=n_actions,
    gamma=0.99,
    theta=1e-6
)

# Resolver
pi_results = pi_solver.solve(
    transition_probs=transition_probs,
    rewards=rewards,
    max_iterations=100
)

### 4.5 Visualizar Resultados

In [ ]:
def visualize_policy_and_values(policy, values, grid_size=4, title=""):
    """
    Visualiza la política y función de valor en un GridWorld.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Símbolos de acciones
    action_symbols = ['↑', '→', '↓', '←']
    
    # Crear grids para visualización
    policy_grid = policy.reshape(grid_size, grid_size)
    value_grid = values.reshape(grid_size, grid_size)
    
    # 1. Visualizar Política
    ax1.set_title(f'Política Óptima {title}', fontsize=14, fontweight='bold')
    ax1.set_xlim(-0.5, grid_size - 0.5)
    ax1.set_ylim(-0.5, grid_size - 0.5)
    ax1.set_xticks(range(grid_size))
    ax1.set_yticks(range(grid_size))
    ax1.grid(True, linewidth=2)
    ax1.invert_yaxis()
    
    for i in range(grid_size):
        for j in range(grid_size):
            action = policy_grid[i, j]
            ax1.text(j, i, action_symbols[action], 
                    ha='center', va='center', fontsize=24,
                    color='blue' if (i == grid_size-1 and j == grid_size-1) else 'black')
    
    # Marcar inicio y meta
    ax1.text(0, -0.8, 'INICIO', ha='center', fontsize=10, color='green', fontweight='bold')
    ax1.text(grid_size-1, grid_size-0.2, 'META', ha='center', fontsize=10, color='red', fontweight='bold')
    
    # 2. Visualizar Valores
    im = ax2.imshow(value_grid, cmap='RdYlGn', interpolation='nearest')
    ax2.set_title(f'Función de Valor {title}', fontsize=14, fontweight='bold')
    ax2.set_xticks(range(grid_size))
    ax2.set_yticks(range(grid_size))
    
    # Añadir valores numéricos
    for i in range(grid_size):
        for j in range(grid_size):
            text = ax2.text(j, i, f'{value_grid[i, j]:.3f}',
                          ha='center', va='center', color='black', fontsize=10)
    
    plt.colorbar(im, ax=ax2, label='Valor del Estado')
    plt.tight_layout()
    plt.show()

# Visualizar resultados de Policy Iteration
visualize_policy_and_values(
    pi_results['policy'], 
    pi_results['V'], 
    grid_size=4,
    title="(Policy Iteration)"
)

### 4.6 Análisis de Convergencia

In [ ]:
# Gráfico de cambios en la política
plt.figure(figsize=(10, 5))
plt.plot(pi_results['history']['policy_changes'], 'o-', linewidth=2, markersize=8)
plt.xlabel('Iteración', fontsize=12)
plt.ylabel('Número de Estados que Cambiaron', fontsize=12)
plt.title('Convergencia de Policy Iteration', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Estadísticas de Policy Iteration:")
print(f"   - Iteraciones totales: {pi_results['iterations']}")
print(f"   - Valor medio final: {np.mean(pi_results['V']):.4f}")
print(f"   - Valor máximo: {np.max(pi_results['V']):.4f}")
print(f"   - Valor mínimo: {np.min(pi_results['V']):.4f}")

## 5. Value Iteration

### 5.1 Algoritmo

Value Iteration combina policy evaluation y policy improvement en un solo paso:

**Algoritmo:**
```
Inicializar V(s) = 0 para todo s

Repetir hasta convergencia:
    Para cada estado s:
        V(s) ← max_a Σ_{s',r} p(s',r|s,a)[r + γV(s')]

Extraer política:
    π(s) = argmax_a Σ_{s',r} p(s',r|s,a)[r + γV(s')]
```

**Diferencias clave con Policy Iteration:**
- No mantiene una política explícita durante el proceso
- Hace actualizaciones más simples pero más frecuentes
- Generalmente más rápido para problemas grandes
- La política solo se extrae al final

### 5.2 Implementación

In [ ]:
print("\n" + "="*50)
print("EJECUTANDO VALUE ITERATION")
print("="*50 + "\n")

# Crear solver
vi_solver = ValueIteration(
    n_states=n_states,
    n_actions=n_actions,
    gamma=0.99,
    theta=1e-6
)

# Resolver
vi_results = vi_solver.solve(
    transition_probs=transition_probs,
    rewards=rewards,
    max_iterations=1000,
    verbose=True
)

### 5.3 Visualizar Resultados

In [ ]:
# Visualizar resultados de Value Iteration
visualize_policy_and_values(
    vi_results['policy'], 
    vi_results['V'], 
    grid_size=4,
    title="(Value Iteration)"
)

### 5.4 Análisis de Convergencia

In [ ]:
# Gráficos de convergencia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Delta máximo
ax1.plot(vi_results['history']['max_deltas'], linewidth=2)
ax1.set_xlabel('Iteración', fontsize=12)
ax1.set_ylabel('Delta Máximo', fontsize=12)
ax1.set_title('Convergencia de Value Iteration (Delta)', fontsize=14, fontweight='bold')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Valor medio
ax2.plot(vi_results['history']['mean_values'], linewidth=2, color='green')
ax2.set_xlabel('Iteración', fontsize=12)
ax2.set_ylabel('Valor Medio', fontsize=12)
ax2.set_title('Evolución del Valor Medio', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Estadísticas de Value Iteration:")
print(f"   - Iteraciones totales: {vi_results['iterations']}")
print(f"   - Delta final: {vi_results['history']['max_deltas'][-1]:.8f}")
print(f"   - Valor medio final: {np.mean(vi_results['V']):.4f}")

## 6. Comparación de Algoritmos

### 6.1 Comparación Teórica

| Aspecto | Policy Iteration | Value Iteration |
|---------|-----------------|----------------|
| **Convergencia** | Menos iteraciones | Más iteraciones |
| **Costo por iteración** | Alto (policy evaluation completo) | Bajo (un solo sweep) |
| **Tiempo total** | Variable | Generalmente más rápido |
| **Política durante proceso** | Siempre disponible | Solo al final |
| **Complejidad** | O(mn² + n²) por iteración | O(mn²) por iteración |
| **Mejor para** | Espacios pequeños, políticas iniciales buenas | Espacios grandes, sin conocimiento previo |

Donde: n = número de estados, m = número de acciones

### 6.2 Comparación Empírica

In [ ]:
# Comparación lado a lado
print("="*70)
print("COMPARACIÓN: POLICY ITERATION vs VALUE ITERATION")
print("="*70)

comparison_data = {
    'Métrica': [
        'Iteraciones',
        'Valor medio final',
        'Diferencia máxima en V',
        'Diferencia en política'
    ],
    'Policy Iteration': [
        pi_results['iterations'],
        f"{np.mean(pi_results['V']):.6f}",
        '-',
        '-'
    ],
    'Value Iteration': [
        vi_results['iterations'],
        f"{np.mean(vi_results['V']):.6f}",
        f"{np.max(np.abs(pi_results['V'] - vi_results['V'])):.8f}",
        f"{np.sum(pi_results['policy'] != vi_results['policy'])} estados"
    ]
}

import pandas as pd
df_comparison = pd.DataFrame(comparison_data)
print(df_comparison.to_string(index=False))

print("\n" + "="*70)
print("CONCLUSIONES:")
print("="*70)
print("✓ Ambos algoritmos convergen a la misma solución óptima")
print(f"✓ Policy Iteration: {pi_results['iterations']} iteraciones")
print(f"✓ Value Iteration: {vi_results['iterations']} iteraciones")
print(f"✓ Diferencia máxima en valores: {np.max(np.abs(pi_results['V'] - vi_results['V'])):.8f}")

### 6.3 Visualización Comparativa

In [ ]:
# Comparación de funciones de valor
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Policy Iteration
im1 = axes[0].imshow(pi_results['V'].reshape(4, 4), cmap='RdYlGn', interpolation='nearest')
axes[0].set_title('Policy Iteration', fontsize=14, fontweight='bold')
plt.colorbar(im1, ax=axes[0])

# Value Iteration
im2 = axes[1].imshow(vi_results['V'].reshape(4, 4), cmap='RdYlGn', interpolation='nearest')
axes[1].set_title('Value Iteration', fontsize=14, fontweight='bold')
plt.colorbar(im2, ax=axes[1])

# Diferencia
diff = np.abs(pi_results['V'] - vi_results['V']).reshape(4, 4)
im3 = axes[2].imshow(diff, cmap='Reds', interpolation='nearest')
axes[2].set_title('Diferencia Absoluta', fontsize=14, fontweight='bold')
plt.colorbar(im3, ax=axes[2])

for ax in axes:
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))

plt.tight_layout()
plt.show()

## 7. Experimentos y Casos de Prueba

### 7.1 Experimento 1: Efecto del Factor de Descuento (γ)

El factor de descuento $\gamma$ controla cuánto valora el agente las recompensas futuras:
- $\gamma \approx 0$: Agente miope (solo recompensas inmediatas)
- $\gamma \approx 1$: Agente previsor (valora mucho el futuro)

In [ ]:
print("="*70)
print("EXPERIMENTO 1: Efecto del Factor de Descuento (γ)")
print("="*70 + "\n")

gammas = [0.5, 0.9, 0.99, 0.999]
results_gamma = {}

for gamma in gammas:
    print(f"\nProbando γ = {gamma}...")
    solver = ValueIteration(
        n_states=n_states,
        n_actions=n_actions,
        gamma=gamma,
        theta=1e-6
    )
    result = solver.solve(transition_probs, rewards, verbose=False)
    results_gamma[gamma] = result
    print(f"  ✓ Iteraciones: {result['iterations']}, Valor medio: {np.mean(result['V']):.4f}")

# Visualizar efecto de gamma
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, gamma in enumerate(gammas):
    value_grid = results_gamma[gamma]['V'].reshape(4, 4)
    im = axes[idx].imshow(value_grid, cmap='RdYlGn', interpolation='nearest')
    axes[idx].set_title(f'γ = {gamma} (iter={results_gamma[gamma]["iterations"]})', 
                       fontsize=12, fontweight='bold')
    
    # Añadir valores
    for i in range(4):
        for j in range(4):
            axes[idx].text(j, i, f'{value_grid[i, j]:.2f}',
                         ha='center', va='center', color='black', fontsize=9)
    
    plt.colorbar(im, ax=axes[idx])
    axes[idx].set_xticks(range(4))
    axes[idx].set_yticks(range(4))

plt.tight_layout()
plt.show()

### 7.2 Experimento 2: GridWorlds de Diferentes Tamaños

In [ ]:
print("\n" + "="*70)
print("EXPERIMENTO 2: Escalabilidad con Tamaño del Grid")
print("="*70 + "\n")

grid_sizes = [3, 4, 5, 6]
results_sizes = {'PI': [], 'VI': []}

for size in grid_sizes:
    print(f"\nGrid {size}x{size} ({size*size} estados):")
    
    # Crear MDP
    trans, rew, n_s, n_a = create_gridworld_mdp(grid_size=size)
    
    # Policy Iteration
    pi = PolicyIteration(n_s, n_a, gamma=0.99, theta=1e-6)
    pi_res = pi.solve(trans, rew, max_iterations=100)
    results_sizes['PI'].append(pi_res['iterations'])
    
    # Value Iteration
    vi = ValueIteration(n_s, n_a, gamma=0.99, theta=1e-6)
    vi_res = vi.solve(trans, rew, verbose=False)
    results_sizes['VI'].append(vi_res['iterations'])
    
    print(f"  Policy Iteration: {pi_res['iterations']} iteraciones")
    print(f"  Value Iteration:  {vi_res['iterations']} iteraciones")

# Gráfico de escalabilidad
plt.figure(figsize=(10, 6))
x = [s*s for s in grid_sizes]  # Número de estados
plt.plot(x, results_sizes['PI'], 'o-', linewidth=2, markersize=10, label='Policy Iteration')
plt.plot(x, results_sizes['VI'], 's-', linewidth=2, markersize=10, label='Value Iteration')
plt.xlabel('Número de Estados', fontsize=12)
plt.ylabel('Iteraciones hasta Convergencia', fontsize=12)
plt.title('Escalabilidad de los Algoritmos', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 7.3 Experimento 3: GridWorld con Obstáculos

In [ ]:
def create_gridworld_with_obstacles(grid_size=5, obstacles=None):
    """
    Crea un GridWorld con obstáculos.
    
    obstacles: lista de posiciones (row, col) de obstáculos
    """
    if obstacles is None:
        obstacles = [(1, 1), (2, 2), (3, 1)]
    
    n_states = grid_size * grid_size
    n_actions = 4
    goal_state = n_states - 1
    
    transition_probs = np.zeros((n_states, n_actions, n_states))
    rewards = np.zeros((n_states, n_actions, n_states))
    
    # Convertir obstáculos a estados
    obstacle_states = [r * grid_size + c for r, c in obstacles]
    
    def state_to_pos(state):
        return state // grid_size, state % grid_size
    
    def pos_to_state(row, col):
        return row * grid_size + col
    
    for s in range(n_states):
        # Estados objetivo y obstáculos son absorbentes
        if s == goal_state or s in obstacle_states:
            for a in range(n_actions):
                transition_probs[s, a, s] = 1.0
                rewards[s, a, s] = -1.0 if s in obstacle_states else 0.0
            continue
        
        row, col = state_to_pos(s)
        
        for a in range(n_actions):
            new_row, new_col = row, col
            
            if a == 0:  # arriba
                new_row = max(0, row - 1)
            elif a == 1:  # derecha
                new_col = min(grid_size - 1, col + 1)
            elif a == 2:  # abajo
                new_row = min(grid_size - 1, row + 1)
            elif a == 3:  # izquierda
                new_col = max(0, col - 1)
            
            next_state = pos_to_state(new_row, new_col)
            
            # Si el siguiente estado es un obstáculo, quedarse en el lugar
            if next_state in obstacle_states:
                next_state = s
                transition_probs[s, a, next_state] = 1.0
                rewards[s, a, next_state] = -0.1  # Penalización por chocar
            else:
                transition_probs[s, a, next_state] = 1.0
                if next_state == goal_state:
                    rewards[s, a, next_state] = 1.0
                else:
                    rewards[s, a, next_state] = -0.01
    
    return transition_probs, rewards, n_states, n_actions, obstacle_states

print("\n" + "="*70)
print("EXPERIMENTO 3: GridWorld con Obstáculos")
print("="*70 + "\n")

# Crear grid con obstáculos
trans_obs, rew_obs, n_s_obs, n_a_obs, obstacles = create_gridworld_with_obstacles(
    grid_size=5,
    obstacles=[(1, 2), (2, 2), (3, 2)]  # Pared vertical
)

print(f"Grid: 5x5 con {len(obstacles)} obstáculos")
print(f"Obstáculos en estados: {obstacles}\n")

# Resolver con Value Iteration
vi_obs = ValueIteration(n_s_obs, n_a_obs, gamma=0.99, theta=1e-6)
result_obs = vi_obs.solve(trans_obs, rew_obs, verbose=True)

# Visualizar
def visualize_gridworld_with_obstacles(policy, values, obstacles, grid_size=5):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    action_symbols = ['↑', '→', '↓', '←']
    policy_grid = policy.reshape(grid_size, grid_size)
    value_grid = values.reshape(grid_size, grid_size)
    
    # Política
    ax1.set_title('Política Óptima con Obstáculos', fontsize=14, fontweight='bold')
    ax1.set_xlim(-0.5, grid_size - 0.5)
    ax1.set_ylim(-0.5, grid_size - 0.5)
    ax1.set_xticks(range(grid_size))
    ax1.set_yticks(range(grid_size))
    ax1.grid(True, linewidth=2)
    ax1.invert_yaxis()
    
    for i in range(grid_size):
        for j in range(grid_size):
            state = i * grid_size + j
            if state in obstacles:
                # Dibujar obstáculo
                ax1.add_patch(plt.Rectangle((j-0.4, i-0.4), 0.8, 0.8, 
                                           fill=True, color='black', alpha=0.7))
                ax1.text(j, i, '■', ha='center', va='center', 
                        fontsize=20, color='red')
            else:
                action = policy_grid[i, j]
                color = 'red' if state == grid_size*grid_size-1 else 'blue'
                ax1.text(j, i, action_symbols[action], 
                        ha='center', va='center', fontsize=20, color=color)
    
    # Valores
    # Marcar obstáculos en el mapa de valores
    value_grid_masked = value_grid.copy()
    for obs in obstacles:
        value_grid_masked[obs[0], obs[1]] = np.nan
    
    im = ax2.imshow(value_grid, cmap='RdYlGn', interpolation='nearest')
    ax2.set_title('Función de Valor', fontsize=14, fontweight='bold')
    ax2.set_xticks(range(grid_size))
    ax2.set_yticks(range(grid_size))
    
    for i in range(grid_size):
        for j in range(grid_size):
            state = i * grid_size + j
            if state in obstacles:
                ax2.text(j, i, 'X', ha='center', va='center', 
                        color='red', fontsize=16, fontweight='bold')
            else:
                ax2.text(j, i, f'{value_grid[i, j]:.2f}',
                        ha='center', va='center', color='black', fontsize=9)
    
    plt.colorbar(im, ax=ax2)
    plt.tight_layout()
    plt.show()

visualize_gridworld_with_obstacles(
    result_obs['policy'], 
    result_obs['V'], 
    obstacles,
    grid_size=5
)

## 8. Visualizaciones Avanzadas

### 8.1 Mapa de Calor de Q-Values

In [ ]:
def visualize_q_values(solver, transition_probs, rewards, grid_size=4):
    """
    Visualiza los Q-values para cada acción en cada estado.
    """
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    action_names = ['Arriba (↑)', 'Derecha (→)', 'Abajo (↓)', 'Izquierda (←)']
    
    for action in range(4):
        ax = axes[action // 2, action % 2]
        
        # Calcular Q-values para esta acción en todos los estados
        q_values = np.zeros(grid_size * grid_size)
        for state in range(grid_size * grid_size):
            q_values[state] = solver.get_q_value(
                state, action, transition_probs, rewards
            )
        
        # Reshape y visualizar
        q_grid = q_values.reshape(grid_size, grid_size)
        im = ax.imshow(q_grid, cmap='coolwarm', interpolation='nearest')
        ax.set_title(f'Q-Values: {action_names[action]}', 
                    fontsize=12, fontweight='bold')
        
        # Añadir valores
        for i in range(grid_size):
            for j in range(grid_size):
                ax.text(j, i, f'{q_grid[i, j]:.3f}',
                       ha='center', va='center', color='white', fontsize=9)
        
        plt.colorbar(im, ax=ax)
        ax.set_xticks(range(grid_size))
        ax.set_yticks(range(grid_size))
    
    plt.tight_layout()
    plt.show()

print("\nVisualizando Q-Values para cada acción...\n")
visualize_q_values(vi_solver, transition_probs, rewards, grid_size=4)

### 8.2 Animación de Convergencia (Simulada)

In [ ]:
def show_convergence_snapshots():
    """
    Muestra snapshots de la función de valor durante la convergencia.
    """
    print("Ejecutando Value Iteration con snapshots...\n")
    
    # Crear nuevo solver
    solver = ValueIteration(n_states=16, n_actions=4, gamma=0.99, theta=1e-6)
    
    # Snapshots en iteraciones específicas
    snapshot_iters = [1, 3, 5, 10, 20, 50]
    snapshots = []
    
    for iteration in range(max(snapshot_iters) + 1):
        solver.value_update(transition_probs, rewards)
        if iteration in snapshot_iters:
            snapshots.append((iteration, solver.V.copy()))
    
    # Visualizar snapshots
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, (iter_num, V) in enumerate(snapshots):
        value_grid = V.reshape(4, 4)
        im = axes[idx].imshow(value_grid, cmap='RdYlGn', 
                             interpolation='nearest', vmin=-0.1, vmax=1.0)
        axes[idx].set_title(f'Iteración {iter_num}', fontsize=12, fontweight='bold')
        
        # Añadir valores
        for i in range(4):
            for j in range(4):
                axes[idx].text(j, i, f'{value_grid[i, j]:.2f}',
                             ha='center', va='center', color='black', fontsize=9)
        
        axes[idx].set_xticks(range(4))
        axes[idx].set_yticks(range(4))
    
    plt.tight_layout()
    plt.show()

show_convergence_snapshots()

### 8.3 Trayectorias Óptimas

In [ ]:
def simulate_trajectory(policy, start_state, goal_state, grid_size=4, max_steps=20):
    """
    Simula una trayectoria siguiendo la política óptima.
    """
    trajectory = [start_state]
    current_state = start_state
    
    for _ in range(max_steps):
        if current_state == goal_state:
            break
        
        # Tomar acción según política
        action = policy[current_state]
        
        # Simular transición (determinista en nuestro GridWorld)
        row, col = current_state // grid_size, current_state % grid_size
        
        if action == 0:  # arriba
            row = max(0, row - 1)
        elif action == 1:  # derecha
            col = min(grid_size - 1, col + 1)
        elif action == 2:  # abajo
            row = min(grid_size - 1, row + 1)
        elif action == 3:  # izquierda
            col = max(0, col - 1)
        
        current_state = row * grid_size + col
        trajectory.append(current_state)
    
    return trajectory

def visualize_trajectory(policy, trajectory, grid_size=4):
    """
    Visualiza una trayectoria en el grid.
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    action_symbols = ['↑', '→', '↓', '←']
    policy_grid = policy.reshape(grid_size, grid_size)
    
    ax.set_title('Trayectoria Óptima', fontsize=14, fontweight='bold')
    ax.set_xlim(-0.5, grid_size - 0.5)
    ax.set_ylim(-0.5, grid_size - 0.5)
    ax.set_xticks(range(grid_size))
    ax.set_yticks(range(grid_size))
    ax.grid(True, linewidth=2)
    ax.invert_yaxis()
    
    # Dibujar política
    for i in range(grid_size):
        for j in range(grid_size):
            action = policy_grid[i, j]
            ax.text(j, i, action_symbols[action], 
                   ha='center', va='center', fontsize=20, color='lightgray')
    
    # Dibujar trayectoria
    for idx in range(len(trajectory) - 1):
        s1 = trajectory[idx]
        s2 = trajectory[idx + 1]
        
        r1, c1 = s1 // grid_size, s1 % grid_size
        r2, c2 = s2 // grid_size, s2 % grid_size
        
        # Flecha
        arrow = FancyArrowPatch(
            (c1, r1), (c2, r2),
            arrowstyle='->', mutation_scale=30, linewidth=3,
            color='red', alpha=0.7
        )
        ax.add_patch(arrow)
    
    # Marcar inicio y fin
    start = trajectory[0]
    end = trajectory[-1]
    r_start, c_start = start // grid_size, start % grid_size
    r_end, c_end = end // grid_size, end % grid_size
    
    ax.plot(c_start, r_start, 'go', markersize=20, label='Inicio')
    ax.plot(c_end, r_end, 'r*', markersize=25, label='Meta')
    
    ax.legend(loc='upper left', fontsize=12)
    plt.tight_layout()
    plt.show()

# Simular desde diferentes estados iniciales
print("\nSimulando trayectorias desde diferentes puntos de inicio...\n")

start_states = [0, 4, 8]  # Diferentes puntos de inicio
for start in start_states:
    trajectory = simulate_trajectory(
        vi_results['policy'], 
        start_state=start,
        goal_state=15,
        grid_size=4
    )
    print(f"Inicio: Estado {start} → Trayectoria: {trajectory}")
    print(f"  Longitud: {len(trajectory)-1} pasos\n")

# Visualizar una trayectoria
trajectory_example = simulate_trajectory(vi_results['policy'], 0, 15, 4)
visualize_trajectory(vi_results['policy'], trajectory_example, grid_size=4)

## 9. Ejercicios Prácticos

### Ejercicio 1: Modificar Recompensas

**Objetivo**: Entender cómo las recompensas afectan el comportamiento óptimo.

**Tarea**: 
1. Crea un nuevo GridWorld 4x4 con `step_reward = -0.1` (en lugar de -0.01)
2. Resuelve usando Value Iteration
3. Compara la política y trayectorias con el GridWorld original
4. Explica las diferencias

**Pregunta**: ¿Por qué el agente toma rutas más directas con step_reward más negativo?

In [ ]:
# EJERCICIO 1: Escribe tu código aquí
# Pista: Usa create_gridworld_mdp() con diferentes parámetros

# Tu código aquí...


### Ejercicio 2: GridWorld Estocástico

**Objetivo**: Implementar un entorno con transiciones probabilísticas.

**Tarea**:
1. Modifica la función `create_gridworld_mdp` para que las acciones sean estocásticas:
   - 80% de probabilidad: moverse en la dirección deseada
   - 10% de probabilidad: moverse perpendicular a la izquierda
   - 10% de probabilidad: moverse perpendicular a la derecha
2. Resuelve el MDP estocástico
3. Compara con el caso determinista

**Pregunta**: ¿Cómo cambia la política óptima con incertidumbre?

In [ ]:
# EJERCICIO 2: Escribe tu código aquí
def create_stochastic_gridworld(grid_size=4, p_correct=0.8):
    """
    Crea un GridWorld con transiciones estocásticas.
    
    Completa esta función.
    """
    # Tu código aquí...
    pass


### Ejercicio 3: Comparación de Eficiencia

**Objetivo**: Analizar cuándo usar Policy Iteration vs Value Iteration.

**Tarea**:
1. Implementa una función que mida el tiempo de ejecución de ambos algoritmos
2. Ejecuta experimentos con diferentes tamaños de grid (3x3 hasta 10x10)
3. Grafica tiempo de ejecución vs tamaño del problema
4. Determina en qué casos cada algoritmo es más eficiente

**Pregunta**: ¿Cuál es el punto de cruce donde un algoritmo supera al otro?

In [ ]:
# EJERCICIO 3: Escribe tu código aquí
import time

def benchmark_algorithms(grid_sizes):
    """
    Compara tiempos de ejecución de PI y VI.
    
    Completa esta función.
    """
    # Tu código aquí...
    pass


### Ejercicio 4: Mundo con Múltiples Metas

**Objetivo**: Extender el framework a problemas más complejos.

**Tarea**:
1. Crea un GridWorld 6x6 con dos metas:
   - Meta 1: recompensa +1.0 en posición (2, 5)
   - Meta 2: recompensa +0.5 en posición (4, 1)
2. Resuelve usando Value Iteration
3. Visualiza cuáles estados prefieren qué meta

**Pregunta**: ¿Cómo decide el agente entre múltiples objetivos?

In [ ]:
# EJERCICIO 4: Escribe tu código aquí
def create_multi_goal_gridworld(grid_size=6, goals=None):
    """
    Crea un GridWorld con múltiples metas.
    
    goals: lista de tuplas ((row, col), reward)
    
    Completa esta función.
    """
    # Tu código aquí...
    pass


### Ejercicio 5: Análisis de Sensibilidad

**Objetivo**: Estudiar cómo parámetros afectan la convergencia.

**Tarea**:
1. Ejecuta Value Iteration con diferentes valores de `theta` (umbral de convergencia):
   - theta = [1e-2, 1e-4, 1e-6, 1e-8, 1e-10]
2. Para cada theta, registra:
   - Número de iteraciones
   - Precisión de la política (comparando con theta muy pequeño)
3. Grafica trade-off entre precisión y tiempo de cómputo

**Pregunta**: ¿Cuál es el theta óptimo para uso práctico?

In [ ]:
# EJERCICIO 5: Escribe tu código aquí
thetas = [1e-2, 1e-4, 1e-6, 1e-8, 1e-10]

# Tu código aquí...


## Conclusiones y Próximos Pasos

### Resumen de Conceptos Clave

1. **Programación Dinámica** es la base teórica de RL
2. **Policy Iteration**: Converge en menos iteraciones pero cada iteración es costosa
3. **Value Iteration**: Más iteraciones pero cada una es rápida
4. Ambos algoritmos **garantizan convergencia** a la política óptima
5. **Limitaciones**: Requieren modelo completo del entorno

### Limitaciones de DP

- **Curse of Dimensionality**: Complejidad $O(|S|^2|A|)$
- **Requiere modelo**: En la práctica, rara vez tenemos $p(s',r|s,a)$ exacto
- **Solo estados discretos**: Difícil para espacios continuos

### Próximos Temas

Para superar estas limitaciones, estudiaremos:

1. **Monte Carlo Methods**: Aprender sin modelo usando experiencia
2. **Temporal Difference Learning**: Combinar DP y MC (Q-Learning, SARSA)
3. **Function Approximation**: Manejar espacios de estados grandes
4. **Deep RL**: Usar redes neuronales (DQN, Policy Gradients)

### Referencias

- Sutton & Barto (2018): *Reinforcement Learning: An Introduction* - Capítulo 4
- Bellman, R. (1957): *Dynamic Programming*
- Puterman, M. (1994): *Markov Decision Processes*

---

**¡Felicidades!** Has completado el tutorial de Programación Dinámica en Reinforcement Learning.

Continúa practicando con los ejercicios y experimentando con diferentes entornos.